In [1]:
import warnings

warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from mypipes import *

In [2]:
train_file=r"E:\IIT-K\data\data\loan_data_train.csv"
test_file=r"E:\IIT-K\data\data\loan_data_test.csv"

#train_file=r"D:\IIT-K\data\data\loan_data_train.csv"
#test_file=r"D:\IIT-K\data\data\loan_data_test.csv"

ld_train=pd.read_csv(train_file)
ld_test=pd.read_csv(test_file)

ld_train.head()


,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,79542.0,25000,25000,18.49%,60 months,debt_consolidation,27.56%,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years
1,75473.0,19750,19750,17.27%,60 months,debt_consolidation,13.39%,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years
2,67265.0,2100,2100,14.33%,36 months,major_purchase,3.50%,LA,OWN,1000.00,690-694,13,893,1.0,< 1 year
3,80167.0,28000,28000,16.29%,36 months,credit_card,19.62%,NV,MORTGAGE,7083.33,710-714,12,38194,1.0,10+ years
4,17240.0,24250,17431.82,12.23%,60 months,credit_card,23.79%,OH,MORTGAGE,5833.33,730-734,6,31061,2.0,10+ years


In [3]:
ld_test.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,20093,5000,5000,60 months,moving,12.59%,NY,RENT,4416.67,690-694,13,7686,0,< 1 year
1,62445,18000,18000,60 months,debt_consolidation,4.93%,CA,RENT,5258.50,710-714,6,11596,0,10+ years
2,65248,7200,7200,60 months,debt_consolidation,25.16%,LA,MORTGAGE,3750.00,750-754,13,7283,0,6 years
3,81822,7200,7200,36 months,debt_consolidation,17.27%,NY,MORTGAGE,3416.67,790-794,14,4838,0,10+ years
4,57923,22000,22000,60 months,debt_consolidation,18.28%,MI,MORTGAGE,6083.33,720-724,9,20181,0,8 years


In [4]:
ld_train.sample(10)

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
1124,132.0,12375,275,10.91%,36 months,small_business,16.39%,VA,MORTGAGE,3892.75,690-694,11,8332,2.0,3 years
1688,69724.0,35000,34950,23.76%,60 months,debt_consolidation,24.22%,NY,RENT,7331.25,665-669,16,28687,2.0,4 years
1162,57704.0,9000,9000,15.31%,36 months,credit_card,15.88%,NY,RENT,5000.00,675-679,7,4532,1.0,10+ years
1616,91669.0,21000,21000,11.14%,60 months,debt_consolidation,16.18%,NY,MORTGAGE,7083.33,705-709,8,22298,1.0,9 years
992,51954.0,9550,9550,10.74%,36 months,debt_consolidation,14.33%,AL,MORTGAGE,4000.00,700-704,8,9128,0.0,3 years
1089,31004.0,25450,25425,13.99%,60 months,credit_card,26.38%,NC,RENT,4416.67,720-724,13,31828,2.0,3 years
1019,94092.0,12300,12300,6.03%,36 months,debt_consolidation,7.15%,VA,RENT,6666.67,740-744,10,10976,0.0,10+ years
815,69027.0,16000,16000,21.98%,60 months,home_improvement,10.99%,IL,MORTGAGE,3466.67,665-669,8,13797,1.0,7 years
60,10326.0,15400,9458.33,16.32%,60 months,house,15.17%,VA,RENT,6500.00,695-699,11,9989,1.0,4 years
1217,72589.0,14400,14400,13.11%,36 months,credit_card,10.75%,CA,RENT,5000.00,680-684,13,19772,0.0,8 years


In [5]:
ld_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              2199 non-null   float64
 1   Amount.Requested                2199 non-null   object 
 2   Amount.Funded.By.Investors      2199 non-null   object 
 3   Interest.Rate                   2200 non-null   object 
 4   Loan.Length                     2199 non-null   object 
 5   Loan.Purpose                    2199 non-null   object 
 6   Debt.To.Income.Ratio            2199 non-null   object 
 7   State                           2199 non-null   object 
 8   Home.Ownership                  2199 non-null   object 
 9   Monthly.Income                  2197 non-null   float64
 10  FICO.Range                      2200 non-null   object 
 11  Open.CREDIT.Lines               2196 non-null   object 
 12  Revolving.CREDIT.Balance        21

In [6]:
ld_train["Loan.Length"].value_counts(dropna=False)

36 months    1722
60 months     476
NaN             1
.               1
Name: Loan.Length, dtype: int64

In [7]:
#PYTHON PIPELINE
# dont need : ID , Intereste.Rate

# Amount requested : V1
# Amount.Funded.By.Investors : V2 
# Open.CREDIT.Lines : V3
# Revolving.CREDIT.Balance : V4

# convert it to numeric , and then impute missing values with median

# Debt.To.Income.Ratio : V5

# remove percentage sign , covert to numeric and then impute with median

# Loan.Length ,Loan.Purpose,State,Home.Ownership,Employment.Length
# V6-V10
# create dummies for these, considering frequency cutoff as 20

# Monthly.Income : V11,
# Inquiries.in.the.Last.6.Months :V12

# impute missing values with median

# FICO.Range : V13

# split the column at '-', convert the resulting columns to numeric(say a ,b) , 
# then create new column = (a+b)/2 


In [8]:
# pipe_name=pdPipeline([
#     ('name of the process',call to process),
#     ('name of the process', call to process),
#     .....
# ])

In [9]:
p1=pdPipeline([
    ('var_select',VarSelector(['Amount.Requested','Amount.Funded.By.Investors',
                               'Open.CREDIT.Lines','Revolving.CREDIT.Balance'])),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])

In [10]:
p2=pdPipeline([
    ('var_select',VarSelector(['Debt.To.Income.Ratio'])),
    ('string_clean',string_clean(replace_it='%',replace_with='')),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])

In [11]:
p3=pdPipeline([
    ('var_select',VarSelector(['Loan.Length', 'Loan.Purpose','State','Home.Ownership',
                               'Employment.Length'])),
    ('missing_trt',DataFrameImputer()),
    ('create_dummies',get_dummies_Pipe(20))
])

In [12]:
p4=pdPipeline([
    ('var_select',VarSelector(['Monthly.Income','Inquiries.in.the.Last.6.Months'])),
    ('missing_trt',DataFrameImputer())
])

In [13]:
p5=pdPipeline([
    ('var_select',VarSelector(['FICO.Range'])),
    ('custom_fico',custom_fico()),
    ('missing_trt',DataFrameImputer())
])


In [14]:
data_pipe=FeatureUnion([
    ('obj_to_num',p1),
    ('dtir',p2),
    ('obj_to_dum',p3),
    ('num',p4),
    ('fico',p5)
])

In [15]:
data_pipe.fit(ld_train)

FeatureUnion(transformer_list=[('obj_to_num',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=['Amount.Requested',
                                                                              'Amount.Funded.By.Investors',
                                                                              'Open.CREDIT.Lines',
                                                                              'Revolving.CREDIT.Balance'])),
                                                  ('convert_to_numeric',
                                                   convert_to_numeric()),
                                                  ('missing_trt',
                                                   DataFrameImputer())])),
                               ('dtir',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=...
 

In [16]:
len(data_pipe.get_feature_names())

61

In [17]:
data_pipe.transform(ld_train).shape

(2200, 61)

In [18]:
x_train=pd.DataFrame(data=data_pipe.transform(ld_train),
                    columns=data_pipe.get_feature_names())

In [19]:
x_test=pd.DataFrame(data=data_pipe.transform(ld_test),
                    columns=data_pipe.get_feature_names())

In [20]:
x_train.shape

(2200, 61)

In [21]:
x_test.shape

(300, 61)

In [22]:
x_train.columns

Index(['obj_to_num__Amount.Requested',
       'obj_to_num__Amount.Funded.By.Investors',
       'obj_to_num__Open.CREDIT.Lines', 'obj_to_num__Revolving.CREDIT.Balance',
       'dtir__Debt.To.Income.Ratio', 'obj_to_dum__Loan.Length_36 months',
       'obj_to_dum__Loan.Length_60 months',
       'obj_to_dum__Loan.Purpose_debt_consolidation',
       'obj_to_dum__Loan.Purpose_credit_card',
       'obj_to_dum__Loan.Purpose_other',
       'obj_to_dum__Loan.Purpose_home_improvement',
       'obj_to_dum__Loan.Purpose_major_purchase',
       'obj_to_dum__Loan.Purpose_small_business',
       'obj_to_dum__Loan.Purpose_car', 'obj_to_dum__Loan.Purpose_wedding',
       'obj_to_dum__Loan.Purpose_medical', 'obj_to_dum__Loan.Purpose_moving',
       'obj_to_dum__State_CA', 'obj_to_dum__State_NY', 'obj_to_dum__State_FL',
       'obj_to_dum__State_TX', 'obj_to_dum__State_PA', 'obj_to_dum__State_IL',
       'obj_to_dum__State_GA', 'obj_to_dum__State_NJ', 'obj_to_dum__State_VA',
       'obj_to_dum__State_MA',

In [23]:
x_test.columns

Index(['obj_to_num__Amount.Requested',
       'obj_to_num__Amount.Funded.By.Investors',
       'obj_to_num__Open.CREDIT.Lines', 'obj_to_num__Revolving.CREDIT.Balance',
       'dtir__Debt.To.Income.Ratio', 'obj_to_dum__Loan.Length_36 months',
       'obj_to_dum__Loan.Length_60 months',
       'obj_to_dum__Loan.Purpose_debt_consolidation',
       'obj_to_dum__Loan.Purpose_credit_card',
       'obj_to_dum__Loan.Purpose_other',
       'obj_to_dum__Loan.Purpose_home_improvement',
       'obj_to_dum__Loan.Purpose_major_purchase',
       'obj_to_dum__Loan.Purpose_small_business',
       'obj_to_dum__Loan.Purpose_car', 'obj_to_dum__Loan.Purpose_wedding',
       'obj_to_dum__Loan.Purpose_medical', 'obj_to_dum__Loan.Purpose_moving',
       'obj_to_dum__State_CA', 'obj_to_dum__State_NY', 'obj_to_dum__State_FL',
       'obj_to_dum__State_TX', 'obj_to_dum__State_PA', 'obj_to_dum__State_IL',
       'obj_to_dum__State_GA', 'obj_to_dum__State_NJ', 'obj_to_dum__State_VA',
       'obj_to_dum__State_MA',

In [24]:
#lINEAR REGRESSION
